# main3: portfolio-ready rebuild
Step-by-step refresh toward a cleaner, more reproducible pipeline.

In [1]:
import sys, platform
print(sys.executable)
print(platform.python_version())

/usr/bin/python3
3.12.3


In [2]:
from pathlib import Path
root = Path('.')
data_dir = root / 'Data'
data_original_dir = root / 'Data_original'
print('Data dir exists:', data_dir.exists())
print('Data_original dir exists:', data_original_dir.exists())
if data_dir.exists():
    train_dir = data_dir / 'Train'
    val_dir = data_dir / 'Validation'
    print('Train classes:', sorted([p.name for p in train_dir.iterdir() if p.is_dir()]))
    print('Validation classes:', sorted([p.name for p in val_dir.iterdir() if p.is_dir()]))
if data_original_dir.exists():
    train_dir_o = data_original_dir / 'Train'
    val_dir_o = data_original_dir / 'Validation'
    print('Orig Train classes:', sorted([p.name for p in train_dir_o.iterdir() if p.is_dir()]))
    print('Orig Validation classes:', sorted([p.name for p in val_dir_o.iterdir() if p.is_dir()]))

Data dir exists: True
Data_original dir exists: True
Train classes: ['biryani', 'chapli_kebab', 'chocolate_cake', 'samosa', 'seekh_kebab']
Validation classes: ['biryani', 'chapli_kebab', 'chocolate_cake', 'samosa', 'seekh_kebab']
Orig Train classes: ['biryani', 'chapli_kebab', 'chocolate_cake', 'samosa', 'seekh_kebab']
Orig Validation classes: ['biryani', 'chapli_kebab', 'chocolate_cake', 'samosa', 'seekh_kebab']


In [3]:
from pathlib import Path
from collections import Counter

def count_images(root: Path):
    counts = {}
    if not root.exists():
        return counts
    for cls in sorted([p for p in root.iterdir() if p.is_dir()]):
        counts[cls.name] = len(list(cls.glob('*.jpg')))
    return counts

train_counts = count_images(Path('Data')/'Train')
val_counts = count_images(Path('Data')/'Validation')
orig_train_counts = count_images(Path('Data_original')/'Train')
orig_val_counts = count_images(Path('Data_original')/'Validation')

print('Augmented train counts:', train_counts)
print('Augmented val counts:', val_counts)
print('Original train counts:', orig_train_counts)
print('Original val counts:', orig_val_counts)
print('Total augmented train:', sum(train_counts.values()))
print('Total original train:', sum(orig_train_counts.values()))

Augmented train counts: {'biryani': 95, 'chapli_kebab': 95, 'chocolate_cake': 150, 'samosa': 110, 'seekh_kebab': 95}
Augmented val counts: {'biryani': 5, 'chapli_kebab': 4, 'chocolate_cake': 7, 'samosa': 5, 'seekh_kebab': 4}
Original train counts: {'biryani': 19, 'chapli_kebab': 19, 'chocolate_cake': 30, 'samosa': 22, 'seekh_kebab': 19}
Original val counts: {'biryani': 5, 'chapli_kebab': 4, 'chocolate_cake': 7, 'samosa': 5, 'seekh_kebab': 4}
Total augmented train: 545
Total original train: 109
